In [3]:
import boto3
import pandas as pd
original_data = pd.read_csv("D:/unh_sem_2/Distributed and Scalable Data engineering/final_project/data_files/rating.csv")

In [1]:
!pip install boto3

In [5]:
original_data.head()

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,article,title_sentiment
0,81664,NaN,Forbes,"Elizabeth Brownfield, Contributor, \n Elizabet...",superstar chef yannick alléno brings refined f...,Now open in Mayfair at Four Seasons Hotel Lond...,https://www.forbes.com/sites/elizabethbrownfie...,https://imageio.forbes.com/specials-images/ima...,2023-11-01 03:27:21.000000,"Pavyllon London, at Four Seasons Hotel London ...",Monaco,"pavyllon london, at four seasons hotel london ...",Neutral
1,81667,NaN,CNA,NaN,nice claim top spot in ligue 1 with late win a...,Nice moved into provisional first place in the...,https://www.channelnewsasia.com/sport/nice-cla...,https://onecms-res.cloudinary.com/image/upload...,2023-10-27 21:28:48.000000,Nice moved into provisional first place in the...,Monaco,nice moved into provisional first place in the...,Positive
2,81694,time,Time,Christina Larson / AP,amphibians are the world’s most vulnerable spe...,"The world’s frogs, salamanders, newts, and oth...",https://time.com/6320467/amphibians-most-vulne...,https://api.time.com/wp-content/uploads/2023/1...,2023-10-04 17:36:18.000000,"The worlds frogs, salamanders, newts and other...",Madagascar,"the world’s frogs, salamanders, newts and othe...",Negative
3,81695,NaN,Phys.Org,Sara Schmidt,image: rusty red waters in madagascar,Iron-rich sediment colors the red-orange water...,https://phys.org/news/2023-10-image-rusty-red-...,https://scx2.b-cdn.net/gfx/news/2023/image-rus...,2023-10-31 18:04:02.000000,Iron-rich sediment colors the red-orange water...,Madagascar,iron-rich sediment colors the red-orange water...,Neutral
4,81703,NaN,Digital Trends,Jason Struss,everything leaving max (formerly hbo max) in n...,From Gangs of London to Fear the Walking Dead ...,https://www.digitaltrends.com/movies/everythin...,https://www.digitaltrends.com/wp-content/uploa...,2023-10-23 23:09:18.000000,"Everything ends. No, I’m not having an existen...",Madagascar,"everything ends. no, i’m not having an existen...",Neutral


In [4]:
import numpy as np
original_data = original_data.drop(columns=["article_id", "source_id", "source_name", "author", "description", "url", "url_to_image", "published_at", "content", "category", "article"])
original_data = original_data.drop_duplicates(subset=['title'], keep='first')
original_data = original_data.dropna(subset=['title'])

In [5]:
original_data["title_sentiment"].value_counts()

Neutral     39202
Negative     8505
Positive     5747
Name: title_sentiment, dtype: int64

## Balancing the data

In [6]:
neutral_rows = original_data[original_data['title_sentiment'] == 'Neutral']
neutral_subset = neutral_rows.head(6000)
positive_rows = original_data[original_data['title_sentiment'] == 'Positive']
negative_rows = original_data[original_data['title_sentiment'] == 'Negative']
data = pd.concat([neutral_subset, positive_rows, negative_rows], ignore_index=True)

In [7]:
data["title_sentiment"].value_counts()

Negative    8505
Neutral     6000
Positive    5747
Name: title_sentiment, dtype: int64

In [8]:
print(data.columns)

Index(['title', 'title_sentiment'], dtype='object')


In [10]:
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Text Pre-processing

Pre-processing is an essential task in Natural Language Processing (NLP) to prepare text data for training. It converts an input of raw text into cleansed tokens as single words or characters. The main pre-processing methods are summarized below:

1.Tokenizing: Splits words into tokens

2.Lemmatizing: Breaks words into their root format

3.Removal of stop words: Takes off unnecessary words such as "the," "he," "she," etc.

4.Removal of punctuations: Takes off unessential word elements such as comma, period, brackets, parenthesis, etc

In [11]:
import spacy
from nltk.corpus import stopwords
import string
import nltk
import nltk.tokenize
from tqdm import tqdm_notebook as tqdm

nltk.download('stopwords')

def preparation_text_data(data):
    """
    This pipeline prepares the text data, conducting the following steps:
    1) Tokenization
    2) Lemmatization
    4) Removal of stopwords
    5) Removal of punctuation
    """
    # initialize spacy object
    nlp = spacy.load('en_core_web_sm')
    # select raw text
    raw_text = data.title.values.tolist()
    # tokenize
    tokenized_text = [[nlp(i.lower().strip())] for i in tqdm(raw_text)]
    #define the punctuations and stop words
    punc = string.punctuation
    stop_words = set(stopwords.words('english'))
    #lemmatize, remove stopwords and punctuationd
    corpus = []
    for doc in tqdm(tokenized_text):
        corpus.append([word.lemma_ for word in doc[0] if (word.lemma_ not in stop_words and word.lemma_ not in punc)])
    # add prepared data to df
    data["title"] = corpus
    return data

data =  preparation_text_data(data)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-11-1c3dfda6a5bd>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tokenized_text = [[nlp(i.lower().strip())] for i in tqdm(raw_text)]


<ipython-input-11-1c3dfda6a5bd>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(tokenized_text):


## Sentiment Label Mapping

In [12]:
sentiment_mapping = {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
}

mask = data['title_sentiment'].isin(["Negative", "Neutral", "Positive"])

# Map the numerical predictions to human-readable labels only for the selected values
data.loc[mask, 'title_sentiment'] = data.loc[mask, 'title_sentiment'].map(sentiment_mapping)

## Text Representation

The second step involves transforming text data into meaningful vectors that can be understood by the ML model. I applied the TF-IDF (Term Frequency(TF) — Inverse Dense Frequency(IDF)), which creates weighted representations for input data based on the word occurrences in the entire corpus.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()

def text_representation(data):
  data['title'] = data['title'].apply(lambda text: " ".join(set(text)))
  X_tfidf = tfidf_vect.fit_transform(data['title'])
  print(X_tfidf.shape)
  #print(tfidf_vect.get_feature_names())
  X_tfidf = pd.DataFrame(X_tfidf.toarray())
  return X_tfidf#apply the TFIDV function


X_tfidf = text_representation(data)

(20252, 21563)


## Model Training

At this stage, we are ready to build and train the ML Logistic Regression model. We split the dataset intotrain and test sets, fit the model, and predict on the data points. The model achieves an accuracy score of 75%.

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X= X_tfidf
y = data['title_sentiment'].astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf= LogisticRegression()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

C:\Users\Asus\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.78      0.87      0.82      2794
           1       0.68      0.61      0.64      1958
           2       0.80      0.75      0.78      1932

    accuracy                           0.76      6684
   macro avg       0.75      0.74      0.75      6684
weighted avg       0.76      0.76      0.76      6684



## Predict on new instance

We can predict on new instance as shown below, we feed the model with a new headline and we predict the label which is negative in our example as we are conveying war and sanctions.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Assuming you have a new text data point
new_text = "texas instruments breaks ground on new 300-mm semiconductor wafer fabrication plant in utah"

# Vectorize the new text using the same TF-IDF vectorizer used for training
new_text_tfidf = tfidf_vect.transform([new_text])

# Make predictions using the trained logistic regression model
prediction = clf.predict(new_text_tfidf)

# Print the prediction
print("Predicted Sentiment:", prediction[0])

Predicted Sentiment: 1


In [25]:
import joblib
joblib.dump(clf, 'model.joblib')

['model.joblib']

In [16]:
import joblib

# Save the trained model to a file
joblib.dump(clf, 'model.joblib')

# Save the TF-IDF vectorizer to a file
joblib.dump(tfidf_vect, 'tfidf_vect.joblib')

['tfidf_vect.joblib']